In [ ]:
!pip install mxnet-cu101
!pip install gluonnlp pandas tqdm
!pip install sentencepiece==0.1.85
!pip install transformers==2.1.1
!pip install torch==1.3.1

     |████████████████████████████████| 710.5MB 26kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 256kB 12.3MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.9.1-cp36-cp36m-linux_x86_64.whl size=470050 sha256=82fc4fab9b8d47f53dea9bde5d2c0fac88afabd0f53d077e8a8f4b8d9b5827d1
  Stored in directory: /root/.cache/pip/wheels/af/60/16/1f8a40e68b85bd9bd7960e91830bca5e40cd113f3220b7e231
Successfully built gluonnlp
     |████████████████████████████████| 1.0MB 13.2MB/s 
     |████████████████████████████████| 317kB 11.9MB/s 
     |████████████████████████████████| 890kB 12.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=816bc04c09ecf282f8bd7f6ea8d4f1ed5c95854715fb21aa0a8955318270fe5c
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacr

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-9xr63d1_
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-9xr63d1_
  Created wheel for kobert: filename=kobert-0.1.1-cp36-none-any.whl size=12871 sha256=0a0a7696846cb178c6b2561ebf351b330ae87ff314ef45e759d33fac4edc17db
  Stored in directory: /tmp/pip-ephem-wheel-cache-ux442lpq/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [ ]:
from transformers import AdamW
from transformers.optimization import WarmupLinearSchedule

In [ ]:
##GPU 사용 시
device = torch.device("cuda:0")

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


###데이터 로딩

In [ ]:
!wget https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
!wget https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1

--2020-06-14 22:50:07--  https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.1, 2620:100:6031:1::a27d:5101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/374ftkec978br3d/ratings_train.txt [following]
--2020-06-14 22:50:07--  https://www.dropbox.com/s/dl/374ftkec978br3d/ratings_train.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc273208c4810517f799c5717c73.dl.dropboxusercontent.com/cd/0/get/A5qrpx0ulpjaxXUSZeqBX3d4-RZ5MYBz95sVi0u9Z58YVSAe7WAknPGVUyV3VwJRtp5DqMAUTEhsrtrrkiqEt40IZYFwVwavy6oXGOFPa4BAhru2oR7IcTGlMYEHFcRxleQ/file?dl=1# [following]
--2020-06-14 22:50:08--  https://uc273208c4810517f799c5717c73.dl.dropboxusercontent.com/cd/0/get/A5qrpx0ulpjaxXUSZeqBX3d4-RZ5MYBz95sVi0u9Z58YVSAe7WAknPGVUyV3VwJRtp5DqMAUTEhsrtrrkiqEt40IZYFwVw

In [ ]:
dataset_train = nlp.data.TSVDataset("ratings_train.txt?dl=1", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("ratings_test.txt?dl=1", field_indices=[1,2], num_discard_samples=1)

###데이터 처리 모듈
  - 토큰화, 패딩, 인코딩
  - 모델 입력 형식 맞추기

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

### 모델 parameter 지정

In [ ]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

### 로드한 데이터를 데이터 처리 모듈에 적용

In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

### 적용할 Classifier

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

### Optimizer와 Scheduler 정의

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = WarmupLinearSchedule(optimizer, warmup_steps=warmup_step, t_total=t_total)

### 정확도 계산 모듈

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

### 학습 및 epoch 마다 test 성능 출력
  - 매 epoch에서 test 성능을 추출하는 것은 overfit 정도를 확인하기 위함이다
  - test 데이터를 적용할 때는, 모델을 evaluation mode로 전환한다

In [ ]:
from pandas import DataFrame as df

In [ ]:
df_test = df(columns = ['Predicted', 'Label'])

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
        if e==4:
          logits = out.detach().cpu().numpy()
          pred_flat = np.argmax(logits, axis=1).flatten()
          new_data = {'Predicted': pred_flat, 'Label': label}
          df_test = df_test.append(new_data, ignore_index=True)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


epoch 1 batch id 1 loss 0.7079352140426636 train acc 0.609375
epoch 1 batch id 201 loss 0.4629184603691101 train acc 0.5819340796019901
epoch 1 batch id 401 loss 0.46800899505615234 train acc 0.6845386533665836
epoch 1 batch id 601 loss 0.5086910128593445 train acc 0.7351809484193012
epoch 1 batch id 801 loss 0.4230979382991791 train acc 0.7642985330836455
epoch 1 batch id 1001 loss 0.2863030433654785 train acc 0.7820772977022977
epoch 1 batch id 1201 loss 0.34245020151138306 train acc 0.7951576810990841
epoch 1 batch id 1401 loss 0.35641855001449585 train acc 0.803968147751606
epoch 1 batch id 1601 loss 0.24645769596099854 train acc 0.8114459712679575
epoch 1 batch id 1801 loss 0.30610984563827515 train acc 0.8175406024430871
epoch 1 batch id 2001 loss 0.2866309583187103 train acc 0.8231353073463268
epoch 1 batch id 2201 loss 0.26281222701072693 train acc 0.8281888914129941

epoch 1 train acc 0.8317757217007964


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



epoch 1 test acc 0.8802349744245525


epoch 2 batch id 1 loss 0.48998743295669556 train acc 0.796875
epoch 2 batch id 201 loss 0.21845383942127228 train acc 0.8798973880597015
epoch 2 batch id 401 loss 0.2754009962081909 train acc 0.881857855361596
epoch 2 batch id 601 loss 0.3756992816925049 train acc 0.8860752911813644
epoch 2 batch id 801 loss 0.31525763869285583 train acc 0.8899227528089888
epoch 2 batch id 1001 loss 0.24529394507408142 train acc 0.8928103146853147
epoch 2 batch id 1201 loss 0.24586939811706543 train acc 0.8949573272273106
epoch 2 batch id 1401 loss 0.1651846319437027 train acc 0.8968036224125625
epoch 2 batch id 1601 loss 0.21393992006778717 train acc 0.8991938632104934
epoch 2 batch id 1801 loss 0.17302659153938293 train acc 0.9011399916712938
epoch 2 batch id 2001 loss 0.27518558502197266 train acc 0.9032358820589705
epoch 2 batch id 2201 loss 0.19455145299434662 train acc 0.9048940822353476

epoch 2 train acc 0.9064610885949943


...


epoch 3 test acc 0.89681905370844


epoch 4 batch id 1 loss 0.48449382185935974 train acc 0.8125
epoch 4 batch id 201 loss 0.08186662197113037 train acc 0.9529695273631841
epoch 4 batch id 401 loss 0.02509327419102192 train acc 0.9551901496259352
epoch 4 batch id 601 loss 0.18969601392745972 train acc 0.9567127703826955
epoch 4 batch id 801 loss 0.12761719524860382 train acc 0.9580992509363296
epoch 4 batch id 1001 loss 0.09466319531202316 train acc 0.9595560689310689
epoch 4 batch id 1201 loss 0.05286441743373871 train acc 0.9611131348875936
epoch 4 batch id 1401 loss 0.08468720316886902 train acc 0.9620471984296931
epoch 4 batch id 1601 loss 0.07119420170783997 train acc 0.9627674109931293
epoch 4 batch id 1801 loss 0.06220283731818199 train acc 0.9634057468073293
epoch 4 batch id 2001 loss 0.08985351026058197 train acc 0.9638852448775612
epoch 4 batch id 2201 loss 0.04261178523302078 train acc 0.9641001249432076

epoch 4 train acc 0.9646148855233219



epoch 4 test acc 0.8954803388746803


epoch 5 batch id 1 loss 0.31300976872444153 train acc 0.90625
epoch 5 batch id 201 loss 0.06138765439391136 train acc 0.9751243781094527
epoch 5 batch id 401 loss 0.08612407743930817 train acc 0.9755299251870324
epoch 5 batch id 601 loss 0.2523981034755707 train acc 0.9749636023294509
epoch 5 batch id 801 loss 0.02580167166888714 train acc 0.975167759051186
epoch 5 batch id 1001 loss 0.06748449802398682 train acc 0.9760239760239761
epoch 5 batch id 1201 loss 0.07427559792995453 train acc 0.9760746253122398
epoch 5 batch id 1401 loss 0.03049805760383606 train acc 0.9764565488936474
epoch 5 batch id 1601 loss 0.06474699079990387 train acc 0.9770065584009994
epoch 5 batch id 1801 loss 0.06880386918783188 train acc 0.9773736812881733
epoch 5 batch id 2001 loss 0.04926580563187599 train acc 0.9777923538230885
epoch 5 batch id 2201 loss 0.026709159836173058 train acc 0.9779006701499319

epoch 5 train acc 0.978082337883959



epoch 5 test acc 0.8967790920716112


<소스 코드 출처>
https://github.com/SKTBrain/KoBERT